<a href="https://colab.research.google.com/github/brierks/Prediccion-gasolina/blob/main/Copia_de_Examen_final_gasolina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción del Precio de la Gasolina — ETL, Modelo y App Streamlit

**Objetivo:**

1. Realizar ETL para transformar los datos a formato tidy.
2. Entrenar un modelo de regresión lineal múltiple para predecir el precio regular usando estado, mes y año.
3. Construir una aplicación en Streamlit para seleccionar estado/mes/año y mostrar la predicción.

_Archivo fuente:_ `Precios_promedio_diarios_y_mensuales_en_estaciones_de_servicio.xlsx` (hoja `Gasolina_Regular`).

In [ ]:
#install streamlit, one time execution
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.3 MB/s eta 0:00:00


In [ ]:
#import streamlit
import streamlit as st

In [ ]:
# PARTE 1: PROCESO ETL
# 1. Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 2. Cargar datos
file_path = "/content/Precios_Gasolina_Regular.csv"
df = pd.read_csv(file_path, encoding="latin1")

In [ ]:
# 3. Vista inicial
print("Columnas disponibles:", df.columns)
print("Dimensiones:", df.shape)
df.head()

Columnas disponibles: Index(['ï»¿ENTIDAD', '2017', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 99', 'Unnamed: 100', 'Unnamed: 101', 'Unnamed: 102',
       'Unnamed: 103', 'Unnamed: 104', 'Unnamed: 105', 'Unnamed: 106',
       'Unnamed: 107', 'Unnamed: 108'],
      dtype='object', length=109)
Dimensiones: (34, 109)


,ï»¿ENTIDAD,2017,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108
0,Mes,ENE,FEB,MAR,ABR,MAY,JUN,JUL,AGO,SEP,...,MAR,ABR,MAY,JUN,JUL,AGO,SEP,OCT,NOV,DIC
1,Nacional,16,15.88,15.78,15.78,15.68,15.57,15.48,15.56,15.79,...,23.78,23.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aguascalientes,16.19,16.18,16.1,16.12,16.03,15.93,15.86,15.92,16.15,...,24.07,23.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Baja California,16.07,16.04,15.85,15.87,15.83,15.81,15.71,15.8,15.98,...,23.35,23.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Baja California Sur,16.11,16.07,16,16.02,15.93,15.83,15.75,15.82,16.04,...,24.45,24.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:


# 1) Intentar leer las dos primeras filas para reconstruir encabezados

def read_header_and_data(path):
    seps = [',',';', '\t', '|']
    for sep in seps:
        try:
            header_rows = pd.read_csv(path, nrows=2, header=None, encoding='latin1', sep=sep, engine='python')
            data = pd.read_csv(path, skiprows=2, header=None, encoding='latin1', sep=sep, engine='python')
            # si tenemos más de 5 columnas es probable que haya leído bien
            if header_rows.shape[1] >= 13:
                print(f"Lectura con separador '{sep}' -> OK ({header_rows.shape[1]} columnas detectadas)")
                return header_rows, data, sep
        except Exception as e:
            # ignora e intenta siguiente separador
            pass
    # lectura sin forzar separador
    header_rows = pd.read_csv(path, nrows=2, header=None, encoding='latin1', engine='python')
    data = pd.read_csv(path, skiprows=2, header=None, encoding='latin1', engine='python')
    print("Lectura fallback sin separador forzado.")
    return header_rows, data, None

header_rows, data, used_sep = read_header_and_data(file_path)

# 2) Construye nombres de columna a partir de las dos filas de encabezado
h0 = header_rows.iloc[0].fillna(method='ffill').astype(str).str.strip()  # años (o "ENTIDAD" en la primera posición)
h1 = header_rows.iloc[1].astype(str).str.strip()                           # "Mes" + "ENE","FEB",...

col_names = []
for top, down in zip(h0, h1):
    top_u = str(top).strip()
    down_u = str(down).strip()
    # Primera columna (ENTIDAD) normalmente tiene top == 'ENTIDAD' y down == 'Mes'
    if top_u.upper() in ['ENTIDAD','ENTIDAD FEDERATIVA','ENTIDAD_FEDERATIVA','ESTADO'] or down_u.upper()=='MES':
        col_names.append('ENTIDAD')
    else:
        # combinación año_mes, p. ej. "2017_ENE"
        col_names.append(f"{top_u}_{down_u}")

# asignar nombres y limpiar columnas totalmente vacías
data.columns = col_names
data = data.dropna(axis=1, how='all')   # eliminar columnas vacías (si existen)

# 3) Transformar a formato largo (tidy)
df_long = data.melt(id_vars=['ENTIDAD'], var_name='año_mes', value_name='precio_raw')

# 4) Separar año y mes
# Algunas columnas pueden no seguir exactamente el patrón '2017_ENE' (por eso hacemos defensivo)
# Si año_mes no contiene '_' lo asignamos a año y mes = None
split = df_long['año_mes'].str.split('_', n=1, expand=True)
split.columns = ['año_part','mes_part']
df_long = pd.concat([df_long, split], axis=1)

# 5) Mapear meses abreviados en español a número
meses_map = {
    'ENE':1,'ENERO':1,
    'FEB':2,'FEBRERO':2,
    'MAR':3,'MARZO':3,
    'ABR':4,'ABRIL':4,
    'MAY':5,'MAYO':5,
    'JUN':6,'JUNIO':6,
    'JUL':7,'JULIO':7,
    'AGO':8,'AGOSTO':8,
    'SEP':9,'SEPT':9,'SEPTIEMBRE':9,
    'OCT':10,'OCTUBRE':10,
    'NOV':11,'NOVIEMBRE':11,
    'DIC':12,'DICIEMBRE':12
}

df_long['mes_part_up'] = df_long['mes_part'].str.upper().str.strip()
df_long['mes_num'] = df_long['mes_part_up'].map(meses_map)

# 6) Limpiar y convertir 'año' (puede venir '2017' o '2017' correctly)
# si año_part contiene texto extra, intentamos extraer dígitos
df_long['año_part_clean'] = df_long['año_part'].astype(str).str.extract(r'(\d{4}|\d{2})', expand=False)

# Normalizar años con 2 dígitos -> 2000+xx (si aplica)
def normalize_año(x):
    try:
        v = int(x)
        if v < 100:  # '17' -> 2017
            return 2000 + v
        return v
    except:
        return None

df_long['año_num'] = df_long['año_part_clean'].apply(normalize_año)

# 7) Limpiar precio y convertir a numérico
# Reemplazar comas por puntos por si acaso, quitar espacios
df_long['precio_clean'] = df_long['precio_raw'].astype(str).str.replace(',', '.').str.strip()
df_long['precio'] = pd.to_numeric(df_long['precio_clean'], errors='coerce')

# 8) Filtrar filas válidas
df_final = df_long.dropna(subset=['ENTIDAD','año_num','mes_num','precio']).copy()

# 9) Renombrar columnas finales y tipos
df_final = df_final[['ENTIDAD','año_num','mes_num','precio']].rename(
    columns={'ENTIDAD':'estado','año_num':'año','mes_num':'mes'}
)
df_final['año'] = df_final['año'].astype(int)
df_final['mes'] = df_final['mes'].astype(int)
df_final['precio'] = df_final['precio'].astype(float)
df_final['estado'] = df_final['estado'].astype(str).str.strip()

# 10) Ordenar y resetear índice
df_final = df_final.sort_values(['estado','año','mes']).reset_index(drop=True)

# Mostrar resultados
print("Filas resultantes (ejemplo):", df_final.shape[0])
display(df_final.head(20))

# Guarda CSV tidy para usar en el modelo
outpath = "precios_gasolina_tidy.csv"
df_final.to_csv(outpath, index=False, encoding='utf-8')
print("Archivo tidy guardado en:", outpath)


Lectura con separador ',' -> OK (109 columnas detectadas)
Filas resultantes (ejemplo): 3300


/tmp/ipython-input-204538132.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  h0 = header_rows.iloc[0].fillna(method='ffill').astype(str).str.strip()  # años (o "ENTIDAD" en la primera posición)


,estado,año,mes,precio
0,Aguascalientes,2017,1,16.19
1,Aguascalientes,2017,2,16.18
2,Aguascalientes,2017,3,16.10
3,Aguascalientes,2017,4,16.12
4,Aguascalientes,2017,5,16.03
5,Aguascalientes,2017,6,15.93
6,Aguascalientes,2017,7,15.86
7,Aguascalientes,2017,8,15.92
8,Aguascalientes,2017,9,16.15
9,Aguascalientes,2017,10,16.28


Archivo tidy guardado en: precios_gasolina_tidy.csv


In [ ]:
# modelo_gasolina.py
# Entrenamiento del modelo de regresión

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# 1) Carga dataset limpio
df = pd.read_csv("precios_gasolina_tidy.csv")

# 2) Variables
X = df[['estado','año','mes']]
y = df['precio']

# 3) Codificar estado
encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = encoder.fit_transform(X[['estado']]).toarray()
X_model = np.concatenate([X_encoded, X[['año','mes']].values], axis=1)

# 4) División train/test
X_train, X_test, y_train, y_test = train_test_split(X_model, y, test_size=0.2, random_state=42)

# 5) Entrenar
model = LinearRegression()
model.fit(X_train, y_train)

# 6) Evaluación
y_pred = model.predict(X_test)
print("Evaluación del modelo:")
print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

# 7) Guardar modelo y encoder
joblib.dump(model, "modelo_gasolina.joblib")
joblib.dump(encoder, "encoder_gasolina.joblib")
print("Modelo y encoder guardados.")



Evaluación del modelo:
MSE: 0.8063570071997403
R²: 0.8841381030560385
Modelo y encoder guardados.


In [ ]:
%%writefile app_gasolina.py
import numpy as np
import streamlit as st
import pandas as pd
import joblib

# --- Configuración de la página ---
st.set_page_config(
    page_title="Predicción Gasolina México",
    page_icon="⛽",
    layout="centered"
)

# --- Estilos CSS personalizados ---
st.markdown("""
    <style>
        .main {
            background-color: #0f1116;
            color: #ffffff;
            font-family: 'Segoe UI', sans-serif;
        }
        .title {
            text-align: center;
            font-size: 2.2em;
            font-weight: bold;
            color: #00c4ff;
            margin-bottom: 0.2em;
        }
        .subtitle {
            text-align: center;
            font-size: 1.0em;
            color: #d1d1d1;
            margin-bottom: 1.2em;
        }
        .stImage > img {
            border-radius: 12px;
        }
        .result-box {
            background: #1c1f26;
            padding: 18px;
            border-radius: 12px;
            text-align: center;
            margin-top: 18px;
            border: 1px solid #00c4ff33;
        }
        .result-price {
            font-size: 1.6em;
            font-weight: bold;
            color: #00ff9f;
        }
    </style>
""", unsafe_allow_html=True)

# --- Título y imagen ---
st.markdown('<div class="title"> Predicción del Precio de la Gasolina en México</div>', unsafe_allow_html=True)
st.markdown('<div class="subtitle">Modelo entrenado con datos de la CRE</div>', unsafe_allow_html=True)

# --- Imagen: se reemplazó use_column_width por use_container_width ---
st.image("Gasolina.jpg", caption="Predicción de precios con base en datos de la CRE", use_container_width=True)

st.markdown("---")

# --- Caching para modelo/encoder y dataset (compatible con distintas versiones de Streamlit) ---
try:
    @st.cache_resource
    def load_model(path):
        return joblib.load(path)

    @st.cache_data
    def load_data(path):
        return pd.read_csv(path)
except Exception:
    # Fallback para versiones antiguas de Streamlit
    @st.cache(allow_output_mutation=True)
    def load_model(path):
        return joblib.load(path)

    @st.cache(allow_output_mutation=True)
    def load_data(path):
        return pd.read_csv(path)

# Cargar recursos
modelo = load_model("modelo_gasolina.joblib")
encoder = load_model("encoder_gasolina.joblib")
df = load_data("precios_gasolina_tidy.csv")

# --- Lista de estados ordenada ---
estados = sorted(df['estado'].unique())

# --- Inputs en columnas ---
st.header("Selecciona los parámetros")
col1, col2, col3 = st.columns([2,1,1])

with col1:
    estado = st.selectbox('Estado:', estados)
with col2:
    año = st.number_input('Año:', min_value=2017, max_value=2030, value=2023, step=1)
with col3:
    mes = st.number_input('Mes:', min_value=1, max_value=12, value=1, step=1)

# --- Transformación y predicción ---
try:
    estado_encoded = encoder.transform([[estado]]).toarray()
except Exception as e:
    st.error("Error al codificar el estado. Revisa el encoder.")
    st.stop()

entrada = np.concatenate([estado_encoded, [[año, mes]]], axis=1)

with st.spinner("Calculando predicción..."):
    prediccion = modelo.predict(entrada)

# --- Resultado ---
st.markdown('<div class="result-box">', unsafe_allow_html=True)
st.subheader(" Precio estimado de la gasolina regular")
st.markdown(f'<div class="result-price">${prediccion[0]:.2f} MXN por litro</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)



Writing app_gasolina.py


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 794ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦

In [ ]:
!streamlit run app_gasolina.py &>/content/logs.txt & npx localtunnel --port 8501

⠙your url is: https://huge-roses-live.loca.lt
^C
